In [5]:
import pandas as pd
import taxoniq # https://github.com/taxoniq/taxoniq

In [ ]:
'''
1. X read in the csv file
2. X append human-readable taxon names
3. X fix the weirdness of bases in the two columns (to get maximum of total_sequence_length and total_alignment_length)
4. X get the proportion of *post-subsampled bases* that map to each taxid
5. X get the percentage of *total bases* that map to each taxid (requires some proportion estimation)
'''

In [60]:
# set the input file names
nt_hits_filepath = "cov_spike_1/tallied_hits_nt.csv"
subsampled_base_count_filepath = "cov_spike_1/subsampled_bases.count"
human_filter_base_count_filepath = "cov_spike_1/human_filtered_bases.count"
input_base_count_filepath = "cov_spike_1/validated_bases.count"

In [6]:
# read in nt hits data using pandas
nt_hits_df = pd.read_csv(nt_hits_filepath)
nt_hits_df.head()

,taxid,level,total_sequence_length,total_alignment_length
0,469,genus,0.0,51769
1,40214,species,0.0,45777
2,694002,genus,324332925.0,35665
3,694003,species,324332925.0,35665
4,562,species,0.0,12892


In [7]:
# test taxoniq
t = taxoniq.Taxon(9606)
t.scientific_name

'Homo sapiens'

In [19]:
# Task #2 -- append human-readable taxon names using taxoniq

# option #1 - full FOR loop
scientific_name = []
for i in nt_hits_df["taxid"]:
    t = taxoniq.Taxon(i)
    scientific_name.append(t.scientific_name)
print(scientific_name)    
nt_hits_df["scientific_name"] = scientific_name

# option #2 - using python list comprehension (similar to lapply in R)
nt_hits_df["scientific_name"] = [taxoniq.Taxon(i).scientific_name for i in nt_hits_df["taxid"]]

['Acinetobacter', 'Acinetobacter johnsonii', 'Betacoronavirus', 'Betacoronavirus 1', 'Escherichia coli', 'Escherichia', 'Tardiphaga', 'Cutibacterium acnes', 'Cutibacterium', 'Tardiphaga robiniae', 'Acinetobacter sp. NEB 394', 'Sphingomonas sp. NIC1', 'Sphingomonas', 'Macaca mulatta polyomavirus 1', 'Betapolyomavirus', 'Ancylobacter', 'Ancylobacter pratisalsi', 'Mastadenovirus', 'Human mastadenovirus C', 'Acinetobacter lwoffii', 'Tardiphaga sp. vice278']


,taxid,level,total_sequence_length,total_alignment_length,scientific_name
0,469,genus,0.0,51769,Acinetobacter
1,40214,species,0.0,45777,Acinetobacter johnsonii
2,694002,genus,324332925.0,35665,Betacoronavirus
3,694003,species,324332925.0,35665,Betacoronavirus 1
4,562,species,0.0,12892,Escherichia coli


In [62]:
nt_hits_df.head()

,taxid,level,total_sequence_length,total_alignment_length,scientific_name,total_bp,proportion_of_subsampled,total_bp_adjusted,proportion_total_bp_adjusted
0,469,genus,0.0,51769,Acinetobacter,51769.0,0.000158,9.502997e+05,0.000056
1,40214,species,0.0,45777,Acinetobacter johnsonii,45777.0,0.000139,8.403073e+05,0.000050
2,694002,genus,324332925.0,35665,Betacoronavirus,324332925.0,0.987924,5.953630e+09,0.351513
3,694003,species,324332925.0,35665,Betacoronavirus 1,324332925.0,0.987924,5.953630e+09,0.351513
4,562,species,0.0,12892,Escherichia coli,12892.0,0.000039,2.366525e+05,0.000014


In [21]:
# Task #3 -- fix the weirdness of bases in the two columns 
#            (get maximum of total_sequence_length and total_alignment_length)
nt_hits_df["total_bp"] = nt_hits_df[["total_sequence_length", "total_alignment_length"]].max(axis=1)

In [37]:
# Task #4 -- get the proportion of *post-subsampled bases* that map to each taxid
subsampled_count = int(open(subsampled_base_count_filepath, 'r').read())
print(subsampled_count)

# NOTE: this is the proportion of non-host reads
nt_hits_df["proportion_of_subsampled"] = nt_hits_df["total_bp"].divide(subsampled_count)

int

In [ ]:
# Logic for Task #5 -- brainstorming how we need to compute the % of total bp
'''
scaling-factor = bp-before-subsampling / bp-after-subsampling = 1000000/500000 = 2 
taxid-count = 10000 bp to sars-cov-2
non-subsampled-taxid-bp-count = scaling-factor * taxid-count
'''

In [56]:
# Task #5 -- get the percentage of *total bases* that map to each taxid (requires some proportion estimation)

# read in the human_filter_count (because this is the step right before subsampling)
human_filter_count = int(open(human_filter_base_count_filepath, 'r').read())
print(human_filter_count)

scaling_factor = human_filter_count / subsampled_count
print(scaling_factor)

# compute scaling-factor adjusted bp counts
nt_hits_df["total_bp_adjusted"] = nt_hits_df["total_bp"].multiply(scaling_factor)

# read in the total initial bp (from validate_input step)
# NOTE: KK realized that this is the incorrect number! we actually need to separately get the
#       total input bp. I pinged Todd about this here: https://czi-sci.slack.com/archives/C03VBBX15UY/p1666213389472059
#       but we can generate this offline for the samples you've already run
total_initial_bp = int(open(input_base_count_filepath, 'r').read())
print(total_initial_bp)

# NOTE: this is proportion of total reads
nt_hits_df["proportion_total_bp_adjusted"] = nt_hits_df["total_bp_adjusted"].divide(total_initial_bp)

6026403345
18.356539160107776


In [49]:
# separate out the dataframes into genus and species level and write to output csv

genus_nt_hits = nt_hits_df[nt_hits_df.level == "genus"]
genus_nt_hits.to_csv("genus_nt_hits.csv")

species_nt_hits = nt_hits_df[nt_hits_df.level == "species"]
species_nt_hits.to_csv("species_nt_hits.csv")